![logo](./img/TheBridge_RL.png)

# Taxi Autónomo (Smartcab), CON RL


## Contenidos

* [Inicializacion](#Inicializacion)  
* [Q-Learning](#Q-Learning)  
* [A programar...](#A-programar...)  


### Inicializacion  
[al indice](#Contenidos)  


In [ ]:
import gym
import warnings


env = gym.make("Taxi-v3", render_mode = "ansi").env

In [ ]:
env.reset(seed = 19)
print(env.render())
print("Current State:", env.s)
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

In [ ]:
movements = [2,0]
for mov in movements:
    env.step(mov)
    print(env.render())
    print("State:",env.s)

In [ ]:
from time import sleep
from IPython.display import clear_output

def episode_animation(frames):
    for i, frame in enumerate(frames): # Recorremos todo el conjunto de frames
        clear_output(wait=True) # Limpiamos la "pantalla"
        print(frame['frame']) # Visualizamos el "pantallazo" resultado de cada acción
        print(f"Timestep: {i + 1}") # Aumentamos el contador de pasos/steps
        # Imprimimos el resto de valores correspondientes a cada frame y que hemos guardado al realizar el "aprendizaje"
        print(f"State: {frame['state']}") 
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Elapsed time (sec.): {frame['elapsed']}")
        sleep(.1) # "Dormimos" el programa un tiempo para que nuestro ojo pueda ver la imagen antes de borrarla y mostrar la siguiente

### Q-Learning  
[al indice](#Contenidos)  


Recordemos brevemente los pasos del algoritmo de Q-Learning epsilon-greedy que nos permitirá estimar la Q-table

<img src="./img/q-matrix-initialized-to-learned_gQq0BFs.png" alt="drawing" width="650"/>

Desglosándolo en pasos, obtenemos:

* Inicializar la tabla Q con todos ceros.
* Seleccionar los valores de los hiperparámetros
* Comenzar a explorar acciones: Para cada estado, seleccione cualquiera entre todas las acciones posibles para el estado actual (S).
* Viajar al siguiente estado (S') como resultado de esa acción (a).
* Para todas las acciones posibles desde el estado (S') seleccione la que tenga el valor Q más alto.
* Actualizar los valores de la tabla Q usando la ecuación ya vista.
* Establecer el siguiente estado como el estado actual.
* Si se alcanza el estado objetivo, entonces terminar y repetir el proceso.


### A programar...  
[al indice](#Contenidos)  


Lo primero creamos la estructura de datos que nos permita almacenar la Q-table

In [ ]:
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

q_table

In [ ]:
q_table.shape

In [ ]:
q_table.size

Lo siguiente es seleccionar los valores de los hiperparámetros, alpha, gamma y épsilon

In [ ]:
alpha = 0.05 #aprendizaje relativamente lento
gamma = 0.9 #darle prioridad a recompensas futuras
epsilon = 0.1 #establecer un 10% de acciones como aleatorias

Ahora podemos crear el algoritmo de entrenamiento que actualizará esta tabla Q a medida que el agente explore el entorno a lo largo de miles de episodios.




In [ ]:
%%time

import random

all_epochs=[]
all_penalties=[]

num_episodes = 100000

state = env.s

for i in range(1,num_episodes+1):
    epochs,penalties,reward = 0,0,0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        next_state,reward,done,truncated,info = env.step(action)
        
        next_max=np.max(q_table[next_state]) #maxQ(S',a')
        old_value = q_table[state,action]
        
        new_value = (1-alpha) * old_value + alpha * (reward + gamma * next_max)
        
        q_table[state,action] = new_value
        
        if reward == -10:
            penalties += 1
        
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output (wait=True)
        print(f"Episode: {i},{i/num_episodes * 100:.2f}")

    state,info = env.reset()
print("Entrenamiento finalizado")

Ahora que hemos estimado la tabla Q tras los 100,000 episodios, veamos cuáles son los valores Q en el estado de nuestra ilustración, que recordemos es el correspondiente al índice 328



In [ ]:
q_table[328]

In [ ]:
q_table

### Evaluación

Vamos a evaluar el rendimiento de nuestro agente. Ya no necesitamos explorar acciones, así que ahora la siguiente acción siempre se selecciona usando el mejor valor Q:


In [ ]:


total_epochs,total_penalties,total_reward = 0,0,0
num_episodes = 100
state = env.s

set_frames = [] #Tendrá un elemento por episodio que contendrá los frames de ese episodio
#e información adicional. 

for i in range(1, num_episodes +1): 
    epochs, penalties, reward = 0,0,0
    done = False
    frames = []
    
    while not done: 
    
        action = np.argmax(q_table[state])
        state, reward, done, truncated, info = env.step(action)
        
        total_reward += reward
        frames.append({
            "frame": env.render(),
            "state":state,
            "action":action,
            "reward":reward,
            "elapsed": 0
        })
        
        if reward == -10:
            penalties += 1
            
        
        epochs += 1
    set_frames.append(frames)
    total_epochs += epochs
    total_penalties += penalties
    state,info = env.reset()
    
print(f"Resultados después de {num_episodes} episodios")
print(f"Numero medio de acciones por episodio: {total_epochs/num_episodes}")
print(f"Numero medio de penalizaciones por episodio: {total_penalties/num_episodes}")
print(f"Recompensa media  por episodio: {total_reward/num_episodes}")

Recuperamos la función de visualización para ver algunos "episodios".

In [ ]:
from time import sleep
from IPython.display import clear_output

def episode_animation(frames):
    for i, frame in enumerate(frames): # Recorremos todo el conjunto de frames
        clear_output(wait=True) # Limpiamos la "pantalla"
        print(frame['frame']) # Visualizamos el "pantallazo" resultado de cada acción
        print(f"Timestep: {i + 1}") # Aumentamos el contador de pasos/steps
        # Imprimimos el resto de valores correspondientes a cada frame y que hemos guardado al realizar el "aprendizaje"
        print(f"State: {frame['state']}") 
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Elapsed time (sec.): {frame['elapsed']}")
        sleep(.1) # "Dormimos" el programa un tiempo para que nuestro ojo pueda ver la imagen antes de borrarla y mostrar la siguiente

Utilicemos ahora la visualización para ver cuanto de bien ha aprendido a conducir. Vamos a analizar 5 episodios escogidos aleatoriamente.

In [ ]:
from random import sample 
from time import sleep 

for frame in sample (set_frames,5):
    episode_animation(frame[0:1])
    sleep(3)
    episode_animation(frame[1:])
    sleep(1)

Bastante bien, ¿no? Comparemos ahora con el "entrenamiento", por llamarlo de alguna forma, sin aprendizaje por refuerzo

### Comparando nuestro agente de Q-learning con no usar Aprendizaje por Refuerzo
  

[al indice](#Contenidos)  



Vamos a evaluar a nuestros agentes de acuerdo con las siguientes métricas,

* Número promedio de penalizaciones por episodio: Cuanto menor sea el número, mejor será el rendimiento de nuestro agente. Idealmente, nos gustaría que esta métrica sea cero o muy cercana a cero.
* Número promedio de pasos por episodio: También queremos que sea un valor pequeño, que nuestro agente tome la ruta más corta para llegar al destino.
* Recompensas promedio por movimiento: Una recompensa más grande significa que el agente está haciendo lo correcto. Es por eso que decidir las recompensas es una parte crucial del Aprendizaje por Refuerzo.


Recuperemos el código que ya desarrollamos en la sesión sin aprendizaje por refuerzo para obtener los valores anteriores para este escenario y hacer la comparativa

In [ ]:
"""Evaluate agent's performance without Q-learning"""

total_epochs, total_penalties, total_rewards = 0, 0, 0
episodes = 100

for _ in range(episodes):
    env.reset()
    # Crea el estado inicial
    state = env.encode(3, 1, 2, 0)
    env.s = state
    # Inicializa las epochs, penalties y rewards
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        # Elige la acción random
        action = env.action_space.sample()
        actions.append(action)
        # Ejecuta la accion
        state, reward, done, truncated, info = env.step(action)
        total_rewards += reward
        # Actualiza el valor de penalties si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per step: {total_rewards/total_epochs}")